In [2]:

import os
import csv
import MDAnalysis
import subprocess
import Bio.PDB
import pandas as pd
import numpy as np
import MDAnalysis.analysis.hbonds
# Naccess for SASA      
# before parse, deal with the pdb files 
path = '/home/tongwade780/test_837_cluster'
outputpath = '/home/tongwade780/test_837_cluster/Naccess_SASA'
files = os.listdir(path)
os.chdir(outputpath)

for onefile in files: 
    if onefile[-3:] == 'pdb': 
        with open(os.path.join(path,onefile),'r') as filetest:
            data = filetest.readlines()
        nonNMR = True
        for line in data:
            if len(line.strip().split()) == 3 and line.strip().split()[-1] == 'NMR':
                NMRfile_MD = MDAnalysis.Universe(os.path.join(path,onefile))
                allatom = NMRfile_MD.select_atoms("all")
                with MDAnalysis.Writer(path+onefile[:-4]+"_1.pdb", allatom.n_atoms) as W:
                    W.write(allatom)
                subprocess.call(["/opt/Naccess/naccess",os.path.join(path,onefile[:-4]+"_1.pdb")])
                nonNMR = False
        if nonNMR and onefile[-6:]!="_1.pdb": 
            subprocess.call(["/opt/Naccess/naccess",os.path.join(path,onefile[:-4]+".pdb")])               

# parse the res files to get the properties of residues 
path ='/home/tongwade780/test_837_cluster/Naccess_SASA'
path_csv = '/home/tongwade780/test_837_cluster/Naccess_SASA/csv'
files = os.listdir(path)
for onefile in files:
    if onefile[-3:] == 'rsa':
        with open(os.path.join(path,onefile), 'r') as naccessData:
            full_data = naccessData.readlines()
            
        with open(os.path.join(path_csv,onefile[:-4]+'_sasa.csv'), 'wb') as output_file:
            output_file_csv = csv.writer(output_file)
            
            for current_line in full_data:
                if current_line[0:3] == 'RES':
                    parsed_line = current_line[10:].strip().split()
                    if current_line[4:7] in ["ALA","CYS","ASP","GLU","PHE","GLY","HIS","ILE","LYS","LEU","MET","ASN","PRO","GLN","ARG","SER","THR","VAL","TRP","TYR"]:
                        output_file_csv.writerow([parsed_line[0], parsed_line[2]])